In [112]:
import pandas as pd
import geopandas as gpd
import json
import statistics

import pickle
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from shapely import Point, LineString

## Lines

In [113]:
with open("data/bus_lines.txt") as f:
    bus_lines = f.read().splitlines()
lines_df = pd.DataFrame(bus_lines, columns=["lineID"])
lines_df.drop_duplicates(inplace=True)
lines_df.to_csv("preprocessed_data/Lines.csv", index=False)


## Stops

In [15]:
stop_gdf = gpd.read_file("data/stops.geojson")
stop_gdf.head()

,id,stop_id,stop_lat,stop_lon,direction,stop_name,direction_id,stop_sequence,route_short_name,geometry
0,453,1780,50.841848,4.364842,0,TRONE,V,0,12,POINT (4.36484 50.84185)
1,454,6433,50.840303,4.367119,0,TRONE,V,1,12,POINT (4.36712 50.84030)
2,455,1131,50.839745,4.373679,0,LUXEMBOURG,V,2,12,POINT (4.37368 50.83975)
3,456,1418,50.843470,4.383977,0,SCHUMAN,V,3,12,POINT (4.38398 50.84347)
4,457,2247,50.853976,4.397888,0,MEISER,V,4,12,POINT (4.39789 50.85398)


In [16]:
stop_gdf = stop_gdf.drop(columns=["id", "stop_lat", "stop_lon", "direction_id"])

In [17]:
def reverse_point_coordinates(geom):
    """
    Reverses the coordinates of Point geometries in the specified geometry column of the DataFrame.

    Parameters:
    df (GeoDataFrame): The GeoDataFrame containing the Point geometries.
    geometry_column (str): The name of the geometry column to process. Default is 'geometry'.

    Returns:
    GeoDataFrame: A GeoDataFrame with the coordinates of Point geometries reversed.
    """
    if geom.geom_type == 'Point':
        x, y = geom.coords[0]
        # Reverse the coordinates
        return Point(y, x)
    return geom

In [18]:
stop_gdf['geometry'] = stop_gdf['geometry'].apply(reverse_point_coordinates)

In [19]:
stop_gdf.rename(columns={'route_short_name': 'line_id'}, inplace=True)

In [20]:
stop_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2189 entries, 0 to 2188
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   stop_id        2189 non-null   int64   
 1   direction      2189 non-null   int64   
 2   stop_name      2189 non-null   object  
 3   stop_sequence  2189 non-null   int64   
 4   line_id        2189 non-null   object  
 5   geometry       2189 non-null   geometry
dtypes: geometry(1), int64(3), object(2)
memory usage: 102.7+ KB


In [21]:
stop_gdf['line_id']= stop_gdf['line_id'].astype(int)

In [22]:
stop_gdf.head()

,stop_id,direction,stop_name,stop_sequence,line_id,geometry
0,1780,0,TRONE,0,12,POINT (50.84185 4.36484)
1,6433,0,TRONE,1,12,POINT (50.84030 4.36712)
2,1131,0,LUXEMBOURG,2,12,POINT (50.83975 4.37368)
3,1418,0,SCHUMAN,3,12,POINT (50.84347 4.38398)
4,2247,0,MEISER,4,12,POINT (50.85398 4.39789)


Primary Key of Stops: stop_id, direction, line_id

In [23]:
stop_gdf.duplicated(subset=['stop_id', 'direction', 'line_id']).sum()

0

In [24]:
stop_gdf.to_file("preprocessed_data/Stops.geojson", driver="GeoJSON")

## Segments

- line_id
- distance
- path (geo)
- direction
- segment_number
- startStopID
- endStopID


In [98]:
segment_gdf = gpd.read_file("data/segments.geojson")
segment_gdf.head()

,id,end,color,start,line_id,distance,direction,geometry
0,1780,6433,#4C8B33,1780,12,0.000000,1,"LINESTRING (4.36479 50.84185, 4.36479 50.84185..."
1,6433,1131,#4C8B33,6433,12,688.862247,1,"LINESTRING (4.36712 50.84033, 4.36713 50.84032..."
2,1131,1418,#4C8B33,1131,12,1734.101558,1,"LINESTRING (4.37362 50.83978, 4.37396 50.84038..."
3,1418,2247,#4C8B33,1418,12,3922.269263,1,"LINESTRING (4.38391 50.84347, 4.38391 50.84347..."
4,2247,4556,#4C8B33,2247,12,6843.167479,1,"LINESTRING (4.39787 50.85398, 4.39803 50.85436..."


In [99]:
segment_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         2061 non-null   object  
 1   end        2061 non-null   int64   
 2   color      2061 non-null   object  
 3   start      2061 non-null   int64   
 4   line_id    2061 non-null   object  
 5   distance   2061 non-null   float64 
 6   direction  2061 non-null   int64   
 7   geometry   2061 non-null   geometry
dtypes: float64(1), geometry(1), int64(3), object(3)
memory usage: 128.9+ KB


In [100]:
segment_gdf['line_id'] = segment_gdf['line_id'].astype(int)

In [101]:
segment_gdf = segment_gdf.drop(columns=["id", "color"])

Stops

- DELTA -> CHIREC = direction 0 (V)
- CHIREC -> DELTA = direction 1 (F)

Segments

- DELTA -> CHIREC = direction 1
- CHIREC -> DELTA = direction 2

In [102]:
# in direction, map 1 to 0 and 2 to 1
segment_gdf["direction"] = segment_gdf["direction"] - 1

In [103]:
# # Step 1: Group by 'line_id' and 'direction', then sort by 'distance'
# segment_gdf = segment_gdf.sort_values(by=['line_id', 'direction', 'distance'])
# # Step 2: Assign a row number to each row within each group
# segment_gdf['segment_number'] = segment_gdf.groupby(['line_id', 'direction']).cumcount()
# segment_gdf.head(10)

In [104]:
segment_gdf.rename(columns={'start': 'start_stop_id', 'end': 'end_stop_id'}, inplace=True)
segment_gdf.head()

,end_stop_id,start_stop_id,line_id,distance,direction,geometry
0,6433,1780,12,0.000000,0,"LINESTRING (4.36479 50.84185, 4.36479 50.84185..."
1,1131,6433,12,688.862247,0,"LINESTRING (4.36712 50.84033, 4.36713 50.84032..."
2,1418,1131,12,1734.101558,0,"LINESTRING (4.37362 50.83978, 4.37396 50.84038..."
3,2247,1418,12,3922.269263,0,"LINESTRING (4.38391 50.84347, 4.38391 50.84347..."
4,4556,2247,12,6843.167479,0,"LINESTRING (4.39787 50.85398, 4.39803 50.85436..."


In [105]:
segment_gdf.drop(columns=["line_id", "direction", "distance"], inplace=True)

In [106]:
print(segment_gdf.shape)

(2061, 3)


In [107]:
segment_gdf.drop_duplicates(subset=["start_stop_id", "end_stop_id"], inplace=True)

In [108]:
print(segment_gdf.shape)

(1680, 3)


In [109]:
def reverse_coordinates(geometry):
    if geometry.geom_type == 'LineString':
        # Reverse the coordinates of each point in the LineString
        reversed_coords = [(y, x) for x, y in geometry.coords]
        return LineString(reversed_coords)
    else:
        # Handle other geometry types if necessary
        return geometry

In [110]:
segment_gdf['geometry'] = segment_gdf['geometry'].apply(reverse_coordinates)

In [111]:
segment_gdf.to_file("preprocessed_data/Segments.geojson", driver="GeoJSON")

## Time

- TimeID
- time
- hour_offset
- minute_offset
- day
- month
- year
- hour
- minute

### MT

In [ ]:
start_date = datetime.strptime("2024-05-23 09:00:00", "%Y-%m-%d %H:%M:%S")
end_date = datetime.strptime("2024-05-23 18:00:00", "%Y-%m-%d %H:%M:%S")

In [ ]:
# Generate a list of datetime values from start_date to end_date at 10-minute intervals
datetime_range = pd.date_range(start=start_date, end=end_date, freq='10T')

# Calculate the hour and minute offsets
hour_offset = (datetime_range - start_date).total_seconds() // 3600
minute_offset = ((datetime_range - start_date).total_seconds() % 3600) // 60


# Create the DataFrame
time_df = pd.DataFrame({
    'datetime': datetime_range,
    'hour_offset': hour_offset,
    'minute_offset': minute_offset,
    'day': datetime_range.day,
    'month': datetime_range.month,
    'year': datetime_range.year,
    'hour': datetime_range.hour,
    'minute': datetime_range.minute
})

time_df.head(10)

### GM

In [40]:
def parse_date(date_str):
    # Parse the datetime string
    parsed_datetime = datetime.fromisoformat(date_str)
    # Convert to the desired format
    formatted_datetime = parsed_datetime.strftime("%Y-%m-%d %H:%M:%S")
    formatted_datetime = datetime.strptime(formatted_datetime, "%Y-%m-%d %H:%M:%S")
    return formatted_datetime

In [41]:
def round_to_nearest_10_minutes(dt):
    """
    Rounds a datetime object to the nearest 10 minutes.

    :param dt: datetime object to be rounded
    :return: datetime object rounded to the nearest 10 minutes
    """
    # Calculate the number of minutes to add or subtract to round to the nearest 10 minutes
    new_minute = (dt.minute // 10) * 10
    remainder = dt.minute % 10

    if remainder >= 5:
        # If the remainder is 5 or more, round up to the next 10 minutes
        dt = dt.replace(minute=new_minute, second=0, microsecond=0) + timedelta(minutes=10)
    else:
        # Otherwise, round down to the nearest 10 minutes
        dt = dt.replace(minute=new_minute, second=0, microsecond=0)
    dt = dt.replace(second=0, microsecond=0)
    return dt

In [42]:
with open("data/gm_segment_speeds_aug27.pkl", "rb") as f:
    gm_segment_speeds = pickle.load(f)

In [43]:
print(len(gm_segment_speeds.keys()))
gm_segment_speeds.keys()

4900


dict_keys([(12, 1, 0, 1), (12, 2, 0, 1), (13, 1, 0, 1), (13, 2, 0, 1), (14, 1, 0, 1), (14, 2, 0, 1), (17, 1, 0, 1), (17, 2, 0, 1), (20, 1, 0, 1), (20, 2, 0, 1), (21, 1, 0, 1), (21, 2, 0, 1), (27, 1, 0, 1), (27, 2, 0, 1), (28, 1, 0, 1), (28, 2, 0, 1), (29, 1, 0, 1), (29, 2, 0, 1), (33, 1, 0, 1), (33, 2, 0, 1), (34, 1, 0, 1), (34, 2, 0, 1), (36, 1, 0, 1), (36, 2, 0, 1), (37, 1, 0, 1), (37, 2, 0, 1), (38, 1, 0, 1), (38, 2, 0, 1), (41, 1, 0, 1), (41, 2, 0, 1), (42, 1, 0, 1), (42, 2, 0, 1), (43, 1, 0, 1), (43, 2, 0, 1), (45, 1, 0, 1), (45, 2, 0, 1), (46, 1, 0, 1), (46, 2, 0, 1), (47, 1, 0, 1), (47, 2, 0, 1), (49, 1, 0, 1), (49, 2, 0, 1), (50, 1, 0, 1), (50, 2, 0, 1), (52, 1, 0, 1), (52, 2, 0, 1), (53, 1, 0, 1), (53, 2, 0, 1), (54, 1, 0, 1), (54, 2, 0, 1), (56, 1, 0, 1), (56, 2, 0, 1), (57, 1, 0, 1), (57, 2, 0, 1), (58, 1, 0, 1), (58, 2, 0, 1), (59, 1, 0, 1), (59, 2, 0, 1), (61, 1, 0, 1), (61, 2, 0, 1), (63, 1, 0, 1), (63, 2, 0, 1), (64, 1, 0, 1), (64, 2, 0, 1), (65, 1, 0, 1), (65, 2, 0, 1),

In [44]:
time_data = []

for key in gm_segment_speeds:
    dt = gm_segment_speeds[key]["time"]
    dt = parse_date(dt)
    dt_bucket = round_to_nearest_10_minutes(dt)

    # Extract additional time components
    day = dt_bucket.day
    month = dt_bucket.month
    year = dt_bucket.year
    hour = dt_bucket.hour
    minute = dt_bucket.minute

    time_data.append({
        "datetime": dt_bucket,
        "day": day,
        "month": month,
        "year": year,
        "hour": hour,
        "minute": minute
    })
time_df = pd.DataFrame(time_data)
time_df.drop_duplicates(inplace=True)
time_df.reset_index(drop=True, inplace=True)
time_df.info()
time_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  53 non-null     datetime64[ns]
 1   day       53 non-null     int64         
 2   month     53 non-null     int64         
 3   year      53 non-null     int64         
 4   hour      53 non-null     int64         
 5   minute    53 non-null     int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 2.6 KB


,datetime,day,month,year,hour,minute
0,2024-08-27 09:00:00,27,8,2024,9,0
1,2024-08-27 09:10:00,27,8,2024,9,10
2,2024-08-27 09:20:00,27,8,2024,9,20
3,2024-08-27 09:30:00,27,8,2024,9,30
4,2024-08-27 09:40:00,27,8,2024,9,40


### STIB

In [47]:
stib_data = pd.read_csv("data/speeds_stib.csv", parse_dates=["datetime"])
stib_data.head()

,start,end,datetime,mean,median
0,1010,1747,2024-08-27 09:00:00,3.821429,4.650
1,1010,1747,2024-08-27 09:10:00,5.512500,5.450
2,1010,1747,2024-08-27 09:20:00,5.450000,5.350
3,1010,1747,2024-08-27 09:30:00,5.000000,5.575
4,1010,1747,2024-08-27 09:40:00,2.664286,2.600


In [49]:
# Extract additional time components
stib_data["day"] = stib_data['datetime'].dt.day
stib_data["month"] = stib_data['datetime'].dt.month
stib_data["year"] = stib_data['datetime'].dt.year
stib_data["hour"] = stib_data['datetime'].dt.hour
stib_data["minute"] = stib_data['datetime'].dt.minute
# remove duplicates
stib_data.drop_duplicates(inplace=True)

In [50]:
# concat with time_df
time_df = pd.concat([time_df, stib_data[["datetime", "day", "month", "year", "hour", "minute"]]])
time_df.drop_duplicates(inplace=True)
# reset index to make surrogate key
time_df.reset_index(inplace=True)
time_df.rename(columns={"index": "id"}, inplace=True)
time_df.to_csv("preprocessed_data/Times.csv", index=False)

## Speeds

- start_stop_id
- end_stop_id
- time_id
- speed
- source (“MT” or “GM” or "STIB")


### MT

In [ ]:
with open("agg_speed_2024-05-23_9am_6pm.json") as f:
    mt_avg_speed_data = json.load(f)

len(mt_avg_speed_data)

In [ ]:
segment_gdf = gpd.read_file("data_tables/Segments.geojson")
segment_gdf.info()

In [ ]:
time_df = pd.read_csv("data_tables/Times.csv", parse_dates=['datetime'])
time_df.info()

In [ ]:
fact_table = []
start_date = datetime.strptime("2024-05-23 09:00:00", "%Y-%m-%d %H:%M:%S")


for i, interval in enumerate(tqdm(mt_avg_speed_data)):
    for entry in interval:
        speed = entry['speed']
        line_id = int(entry['lineId'])
        point_id = int(entry['pointId'])


        condition = (segment_gdf['line_id'] == line_id) & (segment_gdf['start_stop_id'] == point_id)
        if len(segment_gdf[condition]) != 0:
            segment_id = segment_gdf[condition]['id'].values[0]
            time_id = time_df[(time_df['datetime'] == start_date + timedelta(minutes=i*10))]['time_id'].values[0]
            fact_table.append({
                'line_id': line_id,
                'segment_id': segment_id,
                'time_id': time_id,
                'speed': speed,
                'source': 'MT'
            })

In [ ]:
len(fact_table)

### GM

In [53]:
with open("data/gm_segment_speeds_aug27.pkl", "rb") as f:
    gm_avg_speed_data = pickle.load(f)

len(gm_avg_speed_data)

4900

In [54]:
gm_avg_speed_data.keys()

dict_keys([(12, 1, 0, 1), (12, 2, 0, 1), (13, 1, 0, 1), (13, 2, 0, 1), (14, 1, 0, 1), (14, 2, 0, 1), (17, 1, 0, 1), (17, 2, 0, 1), (20, 1, 0, 1), (20, 2, 0, 1), (21, 1, 0, 1), (21, 2, 0, 1), (27, 1, 0, 1), (27, 2, 0, 1), (28, 1, 0, 1), (28, 2, 0, 1), (29, 1, 0, 1), (29, 2, 0, 1), (33, 1, 0, 1), (33, 2, 0, 1), (34, 1, 0, 1), (34, 2, 0, 1), (36, 1, 0, 1), (36, 2, 0, 1), (37, 1, 0, 1), (37, 2, 0, 1), (38, 1, 0, 1), (38, 2, 0, 1), (41, 1, 0, 1), (41, 2, 0, 1), (42, 1, 0, 1), (42, 2, 0, 1), (43, 1, 0, 1), (43, 2, 0, 1), (45, 1, 0, 1), (45, 2, 0, 1), (46, 1, 0, 1), (46, 2, 0, 1), (47, 1, 0, 1), (47, 2, 0, 1), (49, 1, 0, 1), (49, 2, 0, 1), (50, 1, 0, 1), (50, 2, 0, 1), (52, 1, 0, 1), (52, 2, 0, 1), (53, 1, 0, 1), (53, 2, 0, 1), (54, 1, 0, 1), (54, 2, 0, 1), (56, 1, 0, 1), (56, 2, 0, 1), (57, 1, 0, 1), (57, 2, 0, 1), (58, 1, 0, 1), (58, 2, 0, 1), (59, 1, 0, 1), (59, 2, 0, 1), (61, 1, 0, 1), (61, 2, 0, 1), (63, 1, 0, 1), (63, 2, 0, 1), (64, 1, 0, 1), (64, 2, 0, 1), (65, 1, 0, 1), (65, 2, 0, 1),

In [55]:
gm_avg_speed_data[(12, 1, 0, 20)]['speed_data']

[{'distanceMeters': 664, 'duration': '206s', 'speed': 3.2233009708737863},
 {'distanceMeters': 650, 'duration': '164s', 'speed': 3.9634146341463414},
 {'distanceMeters': 1568, 'duration': '394s', 'speed': 3.979695431472081},
 {'distanceMeters': 1755, 'duration': '346s', 'speed': 5.072254335260116},
 {'distanceMeters': 1485, 'duration': '268s', 'speed': 5.541044776119403},
 {'distanceMeters': 1850, 'duration': '268s', 'speed': 6.902985074626866},
 {'distanceMeters': 4925, 'duration': '572s', 'speed': 8.61013986013986},
 {'distanceMeters': 5009, 'duration': '465s', 'speed': 10.772043010752688}]

In [56]:
gdf = gpd.read_file("data/segments.geojson", driver="GeoJSON")
gdf = gdf.drop(columns=["id", "color"])
gdf['line_id'] = gdf['line_id'].astype(int)
gdf['direction'] = gdf['direction']
gdf.head(10)

,end,start,line_id,distance,direction,geometry
0,6433,1780,12,0.000000,1,"LINESTRING (4.36479 50.84185, 4.36479 50.84185..."
1,1131,6433,12,688.862247,1,"LINESTRING (4.36712 50.84033, 4.36713 50.84032..."
2,1418,1131,12,1734.101558,1,"LINESTRING (4.37362 50.83978, 4.37396 50.84038..."
3,2247,1418,12,3922.269263,1,"LINESTRING (4.38391 50.84347, 4.38391 50.84347..."
4,4556,2247,12,6843.167479,1,"LINESTRING (4.39787 50.85398, 4.39803 50.85436..."
5,5044,4556,12,9762.640661,1,"LINESTRING (4.40329 50.86419, 4.40329 50.86419..."
6,3018,5044,12,12750.518888,1,"LINESTRING (4.41472 50.87682, 4.41498 50.87683..."
7,9600,3018,12,15181.898835,1,"LINESTRING (4.43107 50.87851, 4.43128 50.87855..."
8,3017,9600,12,2.338154,2,"LINESTRING (4.48175 50.89819, 4.48196 50.89826..."
9,5048,3017,12,8660.115316,2,"LINESTRING (4.43034 50.87814, 4.43022 50.87811..."


Required format: 

    start_stop_id -> end_stop_id -> datetime = (mean_speed, median_speed)

In [57]:
# already sorted by 'line_id' and 'direction'
grouped = gdf.groupby(['line_id', 'direction'])
groups = grouped.groups
groups[(12, 1)]

Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [58]:
keys = list(gm_avg_speed_data.keys())
gm_avg_speed_data[keys[0]]

{'time': '2024-08-27T09:04:13.583212+02:00',
 'speed_data': [{'distanceMeters': 664,
   'duration': '225s',
   'speed': 2.951111111111111},
  {'distanceMeters': 650, 'duration': '173s', 'speed': 3.7572254335260116},
  {'distanceMeters': 1568, 'duration': '393s', 'speed': 3.989821882951654},
  {'distanceMeters': 1755, 'duration': '420s', 'speed': 4.178571428571429},
  {'distanceMeters': 1485, 'duration': '288s', 'speed': 5.15625},
  {'distanceMeters': 1850, 'duration': '281s', 'speed': 6.5836298932384345},
  {'distanceMeters': 4925, 'duration': '592s', 'speed': 8.319256756756756},
  {'distanceMeters': 5009, 'duration': '471s', 'speed': 10.634819532908704}]}

In [59]:
from collections import defaultdict

# start_stop -> end_stop -> datetime = [speed1, speed2, ...]
segment_speeds_at_time = defaultdict(lambda: defaultdict(lambda : defaultdict(list)))

for key in gm_avg_speed_data:
    line_id, direction, _, _ = key
    time_iso = gm_avg_speed_data[key]['time']
    dt = datetime.fromisoformat(time_iso)
    dt_rounded = round_to_nearest_10_minutes(dt)
    dt_bucket = dt_rounded.strftime("%Y-%m-%d %H:%M:%S")
    speed_data = gm_avg_speed_data[key]['speed_data']
    df_indices = groups[(line_id, direction)]

    assert len(df_indices) == len(speed_data), f"Length mismatch: {len(df_indices)} != {len(speed_data)}. Key: {key}"

    for index, speed_info in zip(df_indices, speed_data):
        start_stop = gdf.loc[index, 'start']
        end_stop = gdf.loc[index, 'end']
        segment_speeds_at_time[start_stop][end_stop][dt_bucket].append(speed_info['speed'])

# compute the average speed for each segment at each time
# start_stop -> end_stop -> datetime = (avg_speed, median_speed)
segment_avg_speeds_at_time =  defaultdict(lambda: defaultdict(lambda : defaultdict(tuple)))

for start_stop in segment_speeds_at_time:
    for end_stop in segment_speeds_at_time[start_stop]:
        for dt in segment_speeds_at_time[start_stop][end_stop]:
            speeds = segment_speeds_at_time[start_stop][end_stop][dt]
            avg_speed = statistics.mean(speeds)
            median_speed = statistics.median(speeds)
            segment_avg_speeds_at_time[start_stop][end_stop][dt] = (avg_speed, median_speed)

In [60]:
def convert_defaultdict_to_dict(d):
    if isinstance(d, defaultdict):
        d = {k: convert_defaultdict_to_dict(v) for k, v in d.items()}
    return d

In [64]:
fact_table = []

for start_stop in segment_avg_speeds_at_time:
    for end_stop in segment_avg_speeds_at_time[start_stop]:
        for dt in segment_avg_speeds_at_time[start_stop][end_stop]:
            avg_speed, median_speed = segment_avg_speeds_at_time[start_stop][end_stop][dt]
            time_id = time_df[(time_df['datetime'] == datetime.strptime(dt, "%Y-%m-%d %H:%M:%S"))].index[0]
            fact_table.append({
                'start_stop_id': start_stop,
                'end_stop_id': end_stop,
                'time_id': time_id,
                'avg_speed': avg_speed,
                'median_speed': median_speed,
                'source': 'GM'
            })
print(len(fact_table))

82620


### STIB

In [63]:
stib_data = pd.read_csv("data/speeds_stib.csv", parse_dates=["datetime"])
stib_data.head()

,start,end,datetime,mean,median
0,1010,1747,2024-08-27 09:00:00,3.821429,4.650
1,1010,1747,2024-08-27 09:10:00,5.512500,5.450
2,1010,1747,2024-08-27 09:20:00,5.450000,5.350
3,1010,1747,2024-08-27 09:30:00,5.000000,5.575
4,1010,1747,2024-08-27 09:40:00,2.664286,2.600


In [67]:
for _, row in tqdm(stib_data.iterrows()):
    time_id = time_df[(time_df['datetime'] == row['datetime'])].index[0]
    fact_table.append({
        'start_stop_id': row['start'],
        'end_stop_id': row['end'],
        'time_id': time_id,
        'avg_speed': row['mean'],
        'median_speed': row['median'],
        'source': 'STIB'
    })
len(fact_table)

0it [00:00, ?it/s]

109680

In [70]:
fact_table_df = pd.DataFrame(fact_table)
fact_table_df.info()
fact_table_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109680 entries, 0 to 109679
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   start_stop_id  109680 non-null  int64  
 1   end_stop_id    109680 non-null  int64  
 2   time_id        109680 non-null  int64  
 3   avg_speed      109680 non-null  float64
 4   median_speed   109680 non-null  float64
 5   source         109680 non-null  object 
dtypes: float64(2), int64(3), object(1)
memory usage: 5.0+ MB


,start_stop_id,end_stop_id,time_id,avg_speed,median_speed,source
0,1780,6433,0,2.951111,2.951111,GM
1,1780,6433,1,3.018182,3.018182,GM
2,1780,6433,2,3.223301,3.223301,GM
3,1780,6433,4,3.239024,3.239024,GM
4,1780,6433,5,3.270936,3.270936,GM


In [71]:
fact_table_df.to_csv("preprocessed_data/FactTable.csv", index=False)